# 创建分段并生成诱发响应（ERP/ERF）

In [1]:
import pathlib
import matplotlib
import mne
import mne_bids

matplotlib.use('Qt5Agg')

In [4]:
sample_data_dir = 'F:/Database/Multimodal_data/mne_data/MNE-sample-data/' # 设置已有数据路径
sample_data_dir = pathlib.Path(sample_data_dir)
bids_root = pathlib.Path(sample_data_dir / 'out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1, h_freq=40) # 带通滤波
events, event_id = mne.events_from_annotations(raw) # 读取事件

Opening raw data file F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_meg.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading events from F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_events.tsv.
Reading channel info from F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_channels.tsv.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamm

In [5]:
event_id

{'Auditory/Left': 1,
 'Auditory/Right': 2,
 'Button': 3,
 'Smiley': 4,
 'Visual/Left': 5,
 'Visual/Right': 6}

## 分段

In [6]:
tmin = -0.3
tmax = 0.5 # 设置epoch的时间窗口
baseline = (None, 0) # 设置基线，用于基线校正

epochs = mne.Epochs(raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    baseline=baseline,
                    preload=True)
epochs

Not setting metadata
320 matching events found
Setting baseline interval to [-0.2996928197375818, 0.0] s
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 320 events and 481 original time points ...
0 bad epochs dropped


Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 s
Baseline,-0.300 – 0.000 s


In [7]:
epochs.plot()

Using matplotlib as 2D backend.


<MNEBrowseFigure size 2560x1351 with 4 Axes>

Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
['MEG 2443', 'EEG 053']


## 根据实验条件选择分段

In [8]:
epochs['Auditory/Right']

Number of events,73
Events,Auditory/Right: 73
Time range,-0.300 – 0.499 s
Baseline,-0.300 – 0.000 s


In [9]:
epochs['Auditory']

Number of events,145
Events,Auditory/Left: 72Auditory/Right: 73
Time range,-0.300 – 0.499 s
Baseline,-0.300 – 0.000 s


In [10]:
epochs['Left']

Number of events,145
Events,Auditory/Left: 72Visual/Left: 73
Time range,-0.300 – 0.499 s
Baseline,-0.300 – 0.000 s


In [11]:
epochs['Visual'].plot_image()

Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
combining channels using "gfp"
combining channels using "gfp"
combining channels using "gfp"


[<Figure size 2560x1448 with 3 Axes>,
 <Figure size 2560x1448 with 3 Axes>,
 <Figure size 2560x1448 with 3 Axes>]

## 保存分段

In [12]:
epochs.save(pathlib.Path(sample_data_dir / 'out_data') / 'epochs_epo.fif', overwrite=True)

## 创建诱发数据

In [13]:
evoked_auditory = epochs['Auditory'].average() # 计算诱发电位
evoked_visual = epochs['Visual'].average()

In [14]:
evoked_auditory.plot(spatial_colors=True) # 画出视觉通道的脑电波

d:\Miniconda3\envs\MF\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
d:\Miniconda3\envs\MF\lib\site-packages\numpy\core\_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


<Figure size 2560x1305 with 6 Axes>

In [15]:
evoked_auditory.plot_topomap(ch_type='mag') # 画出磁通道的拓扑图

<MNEFigure size 600x220 with 5 Axes>

In [16]:
evoked_auditory.plot_joint(picks='mag') # 画出磁通道的脑电波和对应的拓扑图

Projections have already been applied. Setting proj attribute to True.


<Figure size 2560x1448 with 6 Axes>

In [17]:
mne.viz.plot_compare_evokeds([evoked_auditory, evoked_visual], picks='mag') # 画出两个诱发电位的对比

combining channels using "gfp"
combining channels using "gfp"


[<Figure size 2560x1448 with 1 Axes>]

In [18]:
evoked_visual_left = epochs['Visual/Left'].average() # 计算诱发电位
evoked_visual_right = epochs['Visual/Right'].average()
mne.viz.plot_compare_evokeds([evoked_visual_left, evoked_visual_right], picks='eeg')

combining channels using "gfp"
combining channels using "gfp"


[<Figure size 2560x1448 with 1 Axes>]

## 保存诱发数据

In [19]:
mne.write_evokeds(fname=pathlib.Path(sample_data_dir / 'out_data') / 'evokeds_ave.fif', evoked=[evoked_auditory, evoked_visual])

## 读取诱发数据

In [20]:
evokeds = mne.read_evokeds(fname=pathlib.Path(sample_data_dir / 'out_data') / 'evokeds_ave.fif')
evokeds

Reading F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\evokeds_ave.fif ...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
    Found the data of interest:
        t =    -299.69 ...     499.49 ms (0.50 × Auditory/Left + 0.50 × Auditory/Right)
        0 CTF compensation matrices available
        nave = 145 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
Loaded Evoked data is baseline-corrected (baseline: [-0.299693, 0] s)
    Read a total of 3 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
    Found the data of interest:
        t =    -299.69 ...     499.49 ms (0.51 × Visual/Left + 0.49 × Visual/Right)
        0 CTF compensation matrices available
        nave = 144 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
Loaded Evoked data i

[<Evoked | '0.50 × Auditory/Left + 0.50 × Auditory/Right' (average, N=145), -0.29969 – 0.49949 s, baseline -0.299693 – 0 s, 366 ch, ~1.8 MB>,
 <Evoked | '0.51 × Visual/Left + 0.49 × Visual/Right' (average, N=144), -0.29969 – 0.49949 s, baseline -0.299693 – 0 s, 366 ch, ~1.8 MB>]

In [21]:
evokeds[0]

Condition,0.50 × Auditory/Left + 0.50 × Auditory/Right
Data kind,average
Timepoints,481 samples
Channels,366 channels
Number of averaged epochs,145
Time range (secs),-0.2996928095817566 – 0.49948804305179484
Baseline (secs),-0.300 – 0.000 s


In [23]:
evoked = mne.read_evokeds(fname=pathlib.Path(sample_data_dir / 'out_data') / 'evokeds_ave.fif', condition='0.51 × Visual/Left + 0.49 × Visual/Right') # 读取特定的诱发电位
evoked

Reading F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\evokeds_ave.fif ...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
    Found the data of interest:
        t =    -299.69 ...     499.49 ms (0.51 × Visual/Left + 0.49 × Visual/Right)
        0 CTF compensation matrices available
        nave = 144 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
Loaded Evoked data is baseline-corrected (baseline: [-0.299693, 0] s)


Condition,0.51 × Visual/Left + 0.49 × Visual/Right
Data kind,average
Timepoints,481 samples
Channels,366 channels
Number of averaged epochs,144
Time range (secs),-0.2996928095817566 – 0.49948804305179484
Baseline (secs),-0.300 – 0.000 s
